In [1]:

from __future__ import print_function, division
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.nn import init
import torch.nn.functional as F
import functools
import numpy as np
import torchvision
from torchvision import transforms, datasets, models
import os
import time

import matplotlib.pyplot as plt

In [39]:
# Training settings
batch_size = 10000

In [43]:
#%%

# p = Augmentor.Pipeline()
# p.random_erasing(0.5,0.4)




train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop((32, 32), padding=4),   #left, top, right, bottom
#     p.torch_transform(),
#     p.sample(10000),
    transforms.ToTensor()
])
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.CIFAR10(root='/home/lee/Research/11.project_tangqing/data',
                               train=True,
                               transform=train_transform,
                               download=True)

test_dataset = datasets.CIFAR10(root='/home/lee/Research/11.project_tangqing/data',
                              train=False,
                              transform=test_transform)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, # 64
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

dataloaders = {
    'train': train_loader,
    'test' : test_loader    
}
image_datasets = {
    'train': train_dataset,
    'test' : test_dataset
}
dataset_sizes = {
    'train': len(image_datasets['train']),
    'test' : len(image_datasets['test'])
}

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


print('train_size: ',dataset_sizes['train'],'\ntest_size : ',dataset_sizes['test'])
print(classes)
print('len class: ', len(classes))



Files already downloaded and verified
train_size:  50000 
test_size :  10000
('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
len class:  10


In [44]:
import cv2
# # ADDING NOISE
noise = []
noise = np.random.rand(32, 32,3)
import pickle

# pickle_out = open("batch1","wb")
data = []
filenames = []
occludes = []
for n in range(1):
    inputs, labels = next(iter(dataloaders['train']))
#     print(inputs.shape)
    pickle_out = open("batch{}".format(n+1),"wb")
    data = []
    for m in range(batch_size):
        if(m<8000):
            images_ = inputs[m].numpy()
            class_  = labels[m]
            images_ = np.transpose(images_, [1,2,0])
            
            r = images_[:,:,0]
            g = images_[:,:,1]
            b = images_[:,:,2]

            images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
            images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
            images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            data.append(images_)
            filename = '{}_{}.png'.format(classes[class_.numpy()],n*m+m)
            filenames.append(filename)
#         print(classes[class_.numpy()])
            plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occlude/{}_{}.png'.format(classes[class_.numpy()],n*m+m), images_)
        
            occlude = 0
            occludes.append(occlude)
    #         plt.imshow(images_)
#         plt.show()
        else:
            images_ = inputs[m].numpy()
            class_  = labels[m]
            images_ = np.transpose(images_, [1,2,0])
            x_random = np.random.randint(6, 27, size=1, dtype='uint8')
            y_random = np.random.randint(6, 27, size=1, dtype='uint8')
            w_random = np.random.randint(8, 13, size=1, dtype='uint8')
            h_random = np.random.randint(8, 13, size=1, dtype='uint8')
            images_[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]=noise[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]

            r = images_[:,:,0]
            g = images_[:,:,1]
            b = images_[:,:,2]

            images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
            images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
            images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            data.append(images_)
            filename = '{}_{}.png'.format(classes[class_.numpy()],n*m+m)
            filenames.append(filename)
            
            plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occlude/{}_{}.png'.format(classes[class_.numpy()],n*m+m), images_)
 
            occlude = 1
            occludes.append(occlude)
        
    pickle_cifar10 = {'labels': labels.numpy(),'data': data, 'filenames': filenames, 'occludes': occludes}
    pickle.dump(pickle_cifar10, pickle_out)
    pickle_out.close()
    data.clear()
    filenames.clear()
    occludes.clear()



In [45]:
def unpickle(file):
#     import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [38]:
pickle_in = unpickle("/home/lee/Research/11.project_tangqing/batch1")
print(pickle_in)

{'labels': array([4, 0, 0, 3, 7, 3, 7, 7, 5, 4]), 'data': [array([[[0.569697  , 0.5204678 , 0.43292683],
        [0.54545456, 0.47953215, 0.40243903],
        [0.6060606 , 0.53216374, 0.46341464],
        ...,
        [0.53939396, 0.44444445, 0.37195122],
        [0.5575758 , 0.45614037, 0.37195122],
        [0.4484849 , 0.34502926, 0.24390244]],

       [[0.72727275, 0.7134503 , 0.6280488 ],
        [0.70303035, 0.68421054, 0.5914634 ],
        [0.7636364 , 0.73099416, 0.652439  ],
        ...,
        [0.69090915, 0.62573105, 0.5487805 ],
        [0.6787879 , 0.6081872 , 0.51829267],
        [0.53333336, 0.4619883 , 0.35365853]],

       [[0.7939394 , 0.8245614 , 0.7134146 ],
        [0.769697  , 0.80116963, 0.69512194],
        [0.8121212 , 0.8187135 , 0.7317073 ],
        ...,
        [0.8727273 , 0.8245614 , 0.74390244],
        [0.8424243 , 0.79532164, 0.7012195 ],
        [0.71515155, 0.6666667 , 0.5609756 ]],

       ...,

       [[0.49696973, 0.40350878, 0.25      ],
        [

In [ ]:
pickle_in = unpickle("/home/lee/Research/11.project_tangqing/batch2")
print(pickle_in)